## **Preparing pipeline input, Simulating pipeline input**

In [78]:
from google.cloud import storage
import time
import random
import json

In [79]:
storage_client = storage.Client()
bucket = storage_client.bucket('{BUCKET NAME}')

In [80]:
def create_prompt_json(prompt_json,file_name):
    if random.randint(1,1000)%2 == 0:
        prompt_json[file_name] = {}
        prompt_json[file_name]["1"]= [[300, 200]]
    
    return prompt_json

In [81]:
#set the batch size
batch_sz = 5
batch_folder_name = f"batch_{batch_sz}"

In [82]:
prompt_json = {}
# batch_1 is a folder inside sam-pipeline-test bucket
for i in range(batch_sz):
    if (i%2)==0:
        bucket.copy_blob(bucket.blob('pipeline/batch/simple_pencil.jpeg'), bucket, new_name=f'{batch_folder_name}/simple_pencil_{i}.jpeg')
        prompt_json = create_prompt_json(prompt_json, f"simple_pencil_{i}.jpeg")
    else:
        bucket.copy_blob(bucket.blob('pipeline/batch/simple_apple.jpeg'), bucket, new_name=f'{batch_folder_name}/simple_apple_{i}.jpeg')
        prompt_json = create_prompt_json(prompt_json, f"simple_apple_{i}.jpeg")

In [83]:
len(prompt_json)

4

In [84]:
PROMPTS_JSON = f"json_prompts/prompts_json_{batch_sz}.jsonl"
with open(PROMPTS_JSON, "w") as outfile:
    json.dump(prompt_json, outfile)

In [85]:
blob = bucket.blob(f"batch_{batch_sz}/{PROMPTS_JSON}")

blob.upload_from_filename(PROMPTS_JSON)

In [86]:
prompt_json

{'simple_pencil_0.jpeg': {'1': [[300, 200]]},
 'simple_pencil_2.jpeg': {'1': [[300, 200]]},
 'simple_apple_3.jpeg': {'1': [[300, 200]]},
 'simple_pencil_4.jpeg': {'1': [[300, 200]]}}

In [1]:
import tempfile

In [4]:
import time

In [7]:
str(int(time.time()))

'1691459051'